%pylab inline
%precision 6
%load_ext line_profiler
%load_ext autoreload
%autoreload 1

In [ ]:
import pandas as pd
import sklearn as skl
import json
import fasttext

In [ ]:
pd.options.display.max_colwidth=100
np.set_printoptions(linewidth=140,edgeitems=10)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
rcParams['figure.figsize'] = (8.0, 5.0)

In [ ]:
json_lines=[]

with open("tpc-dataset.train.txt",'r') as f:
    for line in f:
        json_lines.append(json.loads(line))

In [ ]:
len(json_lines)

In [ ]:
train_df=pd.DataFrame.from_dict(json_lines)

In [ ]:
train_df.head(3)

In [ ]:
np.random.seed(0)
df_t=train_df
df_t=df_t.sample(frac=1.0).reset_index(drop=True)

In [9]:
df_t.head(3)

,age,education,id,texts
0,18-24,NaN,899434020,"[Это уже все? Или можно вернуть?, Владислав, а если розрабам написать?, Владислав, за 1200-рубле..."
1,NaN,high,326894555,"[Солнце, Форель, Велосипед, Конюхов, Лис, Сок, Ямы, Фингал, Низ, Мясо, Цари, Рог, Нора, Лоб, Нол..."
2,18-24,NaN,703031018,"[Aslan, девочки похлеще пацанов рубятся)), Кристина, я тебя лайкнул,почему ты до сих пор не моя?..."


In [10]:
cleanup_nums = { "age":       {"<=17": 1, "18-24": 2, "25-34":3, "35-44":4, ">=45":5},
                 "education": {"low": 1, "middle": 2,"high": 3} }

In [11]:
df_t.replace(cleanup_nums, inplace=True)
df_t.education = df_t.education.fillna(0)

In [12]:
df_t.head(3)

,age,education,id,texts
0,2.0,0,899434020,"[Это уже все? Или можно вернуть?, Владислав, а если розрабам написать?, Владислав, за 1200-рубле..."
1,NaN,3,326894555,"[Солнце, Форель, Велосипед, Конюхов, Лис, Сок, Ямы, Фингал, Низ, Мясо, Цари, Рог, Нора, Лоб, Нол..."
2,2.0,0,703031018,"[Aslan, девочки похлеще пацанов рубятся)), Кристина, я тебя лайкнул,почему ты до сих пор не моя?..."


In [28]:
df_t.loc[:,'f'] = pd.Series(np.random.randn(len(df_t)), index=df_t.index)
for index, row in df_t.iterrows():
    df_t.at[index, 'f'] =len(df_t.at[index,'texts'])
    df_t.at[index,'text']=''.join(df_t.at[index,'texts'])

In [47]:
Z=df_t.drop(["id","education"],axis=1)
Z.head(3)

,y,texts,f,text
0,18-24,"[Это уже все? Или можно вернуть?, Владислав, а если розрабам написать?, Владислав, за 1200-рубле...",37.0,"Это уже все? Или можно вернуть?Владислав, а если розрабам написать?Владислав, за 1200-рублейПпц,..."
1,NaN,"[Солнце, Форель, Велосипед, Конюхов, Лис, Сок, Ямы, Фингал, Низ, Мясо, Цари, Рог, Нора, Лоб, Нол...",33.0,СолнцеФорельВелосипедКонюховЛисСокЯмыФингалНизМясоЦариРогНораЛобНолик/ноликиИлСанкиСпар (на прав...
2,18-24,"[Aslan, девочки похлеще пацанов рубятся)), Кристина, я тебя лайкнул,почему ты до сих пор не моя?...",303.0,"Aslan, девочки похлеще пацанов рубятся))Кристина, я тебя лайкнул,почему ты до сих пор не моя?😕Па..."


s = df_t.apply(lambda x: pd.Series(x['texts']),axis=1).stack().reset_index(level=1, drop=True)

s.name = 'text'
df_t.drop('texts', axis=1).join(s)
df_t = df_t.reset_index()

res = df_t.set_index(['age', 'education',"id"])['texts'].apply(pd.Series).stack()
res = res.reset_index()
res.columns = ['age','education',"id",'text_num','text']
Z = res.drop('text_num',axis=1)

for index, row in Z.iterrows():
    Z.set_value(index, 'text', Z['text'][index].lower()
                .replace('.', ' ')
                .replace(',', ' ')
                .replace(';', ' ')
                .replace(':', ' ')
                .replace('!', ' ')
                .replace('?', ' ')
                .replace('\r', ' ')
                .replace('\n', ' ')
               )

In [30]:
len(Z)

8607

In [31]:
Z.rename(columns={'age':'y'}, inplace=True)

In [32]:
Z.y

0       2.0
1       NaN
2       2.0
3       3.0
4       NaN
5       1.0
6       3.0
7       NaN
8       NaN
9       1.0
10      5.0
11      2.0
12      4.0
13      NaN
14      1.0
15      2.0
16      NaN
17      1.0
18      1.0
19      NaN
20      2.0
21      1.0
22      NaN
23      3.0
24      3.0
25      NaN
26      3.0
27      1.0
28      NaN
29      2.0
30      1.0
31      3.0
32      NaN
33      2.0
34      NaN
35      3.0
36      2.0
37      3.0
38      2.0
39      2.0
40      3.0
41      3.0
42      2.0
43      NaN
44      3.0
45      4.0
46      NaN
47      1.0
48      3.0
49      NaN
50      1.0
51      NaN
52      2.0
53      2.0
54      1.0
55      3.0
56      NaN
57      1.0
58      3.0
59      NaN
60      1.0
61      4.0
62      3.0
63      1.0
64      NaN
65      2.0
66      3.0
67      2.0
68      3.0
69      1.0
70      1.0
71      3.0
72      3.0
73      1.0
74      NaN
75      3.0
76      1.0
77      5.0
78      3.0
79      3.0
80      1.0
81      3.0
82      3.0
83  

In [33]:
train_sels = ~isnan(Z.y)
test_sels = isnan(Z.y)

train_inds = find(train_sels)
test_inds = find(test_sels)

del train_sels, test_sels

In [34]:
corpus = Z.text

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = TfidfVectorizer(min_df=1)
X = vectorizer.fit_transform(corpus)

Y=Z.y.values

In [35]:
X.shape[0], len(Y)

(8607, 8607)

In [37]:
# Initialize the Random Forest or bagged tree based the model chosen
from sklearn.ensemble import RandomForestClassifier as RFC
rfc = RFC(n_estimators = 100, oob_score = True, max_features = "auto")
print ("Training %s" % ("Random Forest"))
rfc = rfc.fit(X[train_inds], Y[train_inds])
print ("OOB Score =", rfc.oob_score_)
pred = rfc.predict(X[test_inds])


Training Random Forest
OOB Score = 0.463456712673


In [44]:
cleanup_nums = {"age":{ 1:"<=17", 2:"18-24", 3:"25-34", 4:"35-44", 5:">=45"}}


In [45]:
output = pd.DataFrame(data = {"id":test_inds , "age": pred})
output.replace(cleanup_nums, inplace=True)
output.to_csv("submission.csv", index=False)

In [ ]:
import sklearn
from sklearn import svm

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100]}

clf = sklearn.model_selection.GridSearchCV(sklearn.svm.LinearSVC() ,param_grid, n_jobs=-1) # use n_jobs=1 here if have insufficient memory
clf.fit(X[train_inds], Y[train_inds])

In [ ]:
clf.best_params_

In [ ]:
Y_hat = clf.predict(X[test_inds])

In [ ]:
content = concatenate([Z['id'][test_inds].values[:,newaxis], Y_hat[:,newaxis]],axis=1)
content.shape

In [ ]:
solution = pd.DataFrame( content, columns = ['ReviewId', 'Score'], dtype=int)

In [ ]:
solution.head()

In [ ]:
solution.to_csv('d1 solution.csv',index=False)

In [ ]:
model100=fasttext.load_model("fasttext100.bin")